In [1]:
import common
import importlib
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import socket
import matplotlib
import matplotlib.style
import plotting
import matplotlib.ticker as ticker

# Show all columns and rows in a dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

experiments = "(hostname == 'scyper22' or hostname == 'scyper21')"

libraries = ["g1_v6-ref-ignore-build", "livegraph3_ro", "llama8-ref", "sortledton", "stinger7-ref", "teseo-lcc.12"]
libraries_filter = " | ".join(map(lambda l: "library == '" + l + "'", libraries))

edges_per_graph = pd.DataFrame({
    "graph": ["com-friendster", "dota-league", "graph500-22", 
              "graph500-24", "graph500-26", "uniform-22", 
              "uniform-24", "uniform-26"
              ],
    "edges": [1806067135, 50870313, 64155735, 260379520, 1051922853, 64155735, 260379520, 1051922853]
})

scaling_graphs="graph == 'graph500-24'"

## Average Throughput Analysis

In [2]:
data = common.import_gfe("view_updates_throughput").query(experiments) # data from the experiments
throughput_grouped = data.groupby(["graph", "library"]).agg(throughput_mean=("throughput", "mean"))

In [3]:
throughput_pivot = throughput_grouped.pivot_table(values="throughput_mean", columns="library", index="graph")
# Choose newest Sortledton
throughput_pivot["sortledton"] = throughput_pivot["sortledton.3"]
throughput_pivot = throughput_pivot.drop("sortledton.1", axis=1)
throughput_pivot = throughput_pivot.drop("sortledton.3", axis=1)

KeyError: 'sortledton.3'

In [ ]:
## FIGURE 11a
matplotlib.style.use('classic')

colormap=common.set_library_colormap("")

plotting.latexify(fig_width=3)

p = throughput_pivot.plot.bar(color=colormap, legend=None)
p.set_yscale('linear')

plt.xlabel("Dataset")
plt.ylabel("meps")

positions = (0, 1)
x_labels = ("g500-24", "uni-24")
plt.xticks(positions, x_labels)
plt.xticks(rotation=0)

scale_y = 1e6
ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale_y))
p.yaxis.set_major_formatter(ticks_y)
plt.yticks()

plotting.format_axes(p, categorical_x=True, yscale="linear")

plt.tight_layout()
plt.savefig(plotting.base_path + "throughput_average-new.png", dpi=400)

p

In [ ]:
legend_labels = ["GraphOne ($>$10h)", "Livegraph (135m)", "Llama (OOM)", "Sortledton (12m)", "Stinger (6h 40m)", "Teseo (11m)"]
legend = p.legend(ncol=7, borderaxespad=0., labels=legend_labels, fontsize="x-large")
fig  = legend.figure
fig.canvas.draw()
bbox  = legend.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
fig.savefig(plotting.base_path + "legend2.png", dpi=400, bbox_inches=bbox)

## Memory consumption

In [ ]:
memory_consumption_data = common.import_gfe("view_updates_memory_footprint").query(experiments)
memory_consumption_data["memory_usage_gb"] = memory_consumption_data["memory_usage_bytes"] / 1e9
memory_consumption_data["progress"] = memory_consumption_data["progress"].round(2)
grouped = memory_consumption_data.groupby(["graph", "library", "exec_id"]).agg(m=("memory_usage_gb", "max"))
#grouped

In [ ]:
memory_consumption_data_graph500 = memory_consumption_data.query("graph == 'graph500-24'")
memory_consumption_pivot = memory_consumption_data_graph500.pivot_table(values="memory_usage_gb", columns="library", index="progress")
memory_consumption_pivot.interpolate(method="index", inplace=True, limit=1)

memory_consumption_pivot["sortledton"] = memory_consumption_pivot["sortledton.3"]
memory_consumption_pivot = memory_consumption_pivot.drop("sortledton.1", axis=1)
memory_consumption_pivot = memory_consumption_pivot.drop("sortledton.3", axis=1)

In [ ]:
memory_consumption_pivot

In [ ]:
## FIGURE 11c

matplotlib.style.use('classic')

colormap=common.set_library_colormap("")

plotting.latexify(fig_width=3.4)
p= memory_consumption_pivot.plot.line(color=colormap, legend=None)

plt.xlabel("Progress")
plt.ylabel("Memory Consumption [GB]")

plt.annotate("X", (0.42, 204))
plt.annotate("X", (0.54, 51))

plotting.format_axes(p)

plt.tight_layout()
plt.savefig(plotting.base_path + "memory_consumption-new.png", dpi=400)

## Throughput

In [ ]:
throughput_data = common.import_gfe("view_updates_throughput").query(experiments)
throughput_data["minute"] = (throughput_data["second"] / 60).round(1)

In [ ]:
max_minute = throughput_data.groupby("library").aggregate(max_min=("minute", "max"))
max_minute

In [ ]:
pivot_throughput_data_graph500 = throughput_data.query("graph == 'graph500-24'").pivot_table(index="minute", columns="library", values="throughput")
pivot_throughput_data_uniform = throughput_data.query("graph == 'uniform-24'").pivot_table(index="minute", columns="library", values="throughput")

pivot_throughput_data_graph500 = pivot_throughput_data_graph500.query("minute < 45")

# Filter out old version
pivot_throughput_data_graph500["sortledton"] = pivot_throughput_data_graph500["sortledton.3"]
pivot_throughput_data_graph500 = pivot_throughput_data_graph500.drop("sortledton.1", axis=1)
pivot_throughput_data_graph500 = pivot_throughput_data_graph500.drop("sortledton.3", axis=1)
# Filter out experiment end artifacts of Sortledton
pivot_throughput_data_graph500 = pivot_throughput_data_graph500.query("minute!= 12.0 and minute!= 12.4 and minute != 12.3 and minute != 12.2 and minute != 12.1")
pivot_throughput_data_graph500

In [ ]:
## FIGURE 11b

matplotlib.style.use('classic')

colormap=common.set_library_colormap("")

plotting.latexify(fig_width=3)

p = pivot_throughput_data_graph500.plot.line(color=colormap, linewidth=1, legend=None)

p.set_yscale('linear')
scale_y = 1e6

ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale_y))
p.yaxis.set_major_locator(ticker.MultipleLocator(scale_y))
p.yaxis.set_major_formatter(ticks_y)
plt.yticks()

plt.ylabel("meps")
plt.xlabel("Time [minutes]")
plt.xticks(rotation=0)

p.plot(12.3, 3.4e+06, marker="*", markersize=7, color="blue")
p.plot(11.3, 3.7e+06, marker="*", markersize=7, color="mediumorchid")

plotting.format_axes(p, yscale="linear")
plt.grid(axis='y', which="minor", b=None)
           
plt.tight_layout()
plt.savefig(plotting.base_path + "throughput-new.png", dpi=400)